In [ ]:
import numpy as np
import matplotlib.pyplot as plt

filename = 'x64_i5-9600k.txt'
report = ''
with open(filename, 'r') as f:
    report = f.read().split('\n')

prim_cmp = []
q_set = []
p_set = []
d_set = []
c_set = []

for line in report:
    if line == '':
        continue
    cat = line.split('_')
    prim_cmp += [cat[0]]
    q_set += [cat[1]]
    p_set += [cat[2]]
    d_set += [cat[3]]
    c_set += [cat[4].split(';')[0]]

prim_cmp = list(dict.fromkeys(prim_cmp))
q_set = list(dict.fromkeys(q_set))
p_set = list(dict.fromkeys(p_set))
d_set = list(dict.fromkeys(d_set))
c_set = list(dict.fromkeys(c_set))

In [ ]:
from cycler import cycler

# Style configuration
monochrome = (cycler('color', ['k']) * cycler('linestyle', ['-', '--', ':']) * cycler('marker', ['^',',', '.']))
styles = [
          { 'color': 'grey', 'hatch': '...'},
          { 'color': 'white', 'hatch':'\\\\\\'},
          { 'color': 'white', 'hatch':'..'},
          { 'color': 'dimgrey', 'hatch':'xxxx'}
         ]
plt.rcParams['figure.dpi'] = 200
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.prop_cycle'] = monochrome

# returns values [y(px_0), y(px_1), ..., y(px_n)] 
# for given p0 and p1 features
def get_values_y(p0, p1, px):
    res = []
    for p in px:
        res += [[x for x in report if p0 in x and p1 in x and p in x]]  
    ys = []
    for r in res:
        tmp = 0
        for line in r:
            tmp += float(line.split(';')[1]) / len(r)
        ys += [tmp]
    return ys

# ax: rect
# cat: categories for bar chart (i.e. ['q8', 'q16', 'q32'])
# ys: bars per category
# bar_labels: labels of every bar
def plot_bars(ax, cat, ys, bar_labels):
    width = 0.25
    x = np.arange(len(cat))
    for i in range(len(ys)):
        # 0 1 2  - 
        ax.bar(x + (i - len(ys)/2) * width + width / 2, 
               ys[i], width * 0.85, 
               edgecolor = 'black',
               zorder = 10, **styles[i],
               label = bar_labels[i])
        ax.set_xticks(x, [c[1:] for c in cat])

def plot_row(fig, ax, f_0, f_1, f_1_label, f_x):
    for i in range(0, len(ax)):
        ax[i].yaxis.set_major_formatter(lambda x, pos: str(int(x))+'ms')
        ax[i].set_title(f'{f_1[i][1:]} {f_1_label}')

    for idx, f in enumerate(feature_1):
        ys = [get_values_y(_i, f, f_x) for _i in f_0]
        plot_bars(ax[idx], f_x, ys, f_0)
        
    handles, labels = ax[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper right', bbox_to_anchor=(1.12, 0.94))
    fig.tight_layout()
        

# Figure 1
feature_1 = d_set 
fig, ax = plt.subplots(1, len(feature_1), figsize=(3 * len(feature_1), 2.5))
plot_row(fig, ax, prim_cmp, feature_1, 'Dummy Instructions', p_set)



In [ ]:
fig.patch.set_alpha(0.0)
fig.savefig(filename.split('.')[0] + '', bbox_inches='tight')